**Author:** Mattias Östmar, mattiasostmar@gmail.com

**Published:** 2019-03-36

# Create pickled Pandas Dataframe with texts with paragraph separation

This is the initial dataframe created. In `create_dataframe_unparagraphed_texts_from_validation_bypublisher_xml_file` we use a different method to extract the texts from the xml-files in [Hyperpartisan News Detection
PAN @ SemEval 2019](https://pan.webis.de/semeval19/semeval19-web/) where `\n\n` between paragraphs are not preserved. 

In the notebook `compare_paragraphed_vs_non-paragraphed_validation_bypublisher_texts.ipynb` we compare the results from running the discoursebias software on the two different texts, one with `\n\n` preserved (this) and one with no paragraph separations, concluding that the bias index score from both versions of the texts are exactly the same.

In [1]:
import xmltodict
import pandas as pd
import json

In [91]:
# Show the content of subdirectory data/ which should contain the uncompressed dataset
!ls -la data

total 5593256
drwxr-xr-x@ 10 mos  staff         320 21 Mar 13:34 .
drwxr-xr-x@  9 mos  staff         288 21 Mar 13:54 ..
-rw-r--r--@  1 mos  staff        6148 21 Mar 13:34 .DS_Store
-rw-r--r--@  1 mos  staff        2101 21 Mar 09:51 article.xsd
-rw-rw-r--@  1 mos  staff     2718431 16 Nov 15:23 articles-training-byarticle-20181122.xml
-rw-rw-r--@  1 mos  staff  2717573604 22 Nov 01:17 articles-training-bypublisher-20181122.xml
-rw-rw-r--@  1 mos  staff      111875 16 Nov 15:25 ground-truth-training-byarticle-20181122.xml
-rw-rw-r--@  1 mos  staff   104765374 22 Nov 00:13 ground-truth-training-bypublisher-20181122.xml
-rw-rw-r--@  1 mos  staff    25504989 22 Nov 00:12 ground-truth-validation-bypublisher-20181122.xml
-rw-r--r--@  1 mos  staff        1628 21 Mar 09:52 ground-truth.xsd


The below code reads through all the articles in the xml-file `articles-validation-bypublisher-20181122.xml`and stores them in a dataframe. I've left it in the Raw NBConvert-format so that you can inspect it. Change the cell format to `code` if you want to run it all again. The resulting dataframe can be loaded by the code cell below for inspection.

In [115]:
texts_df = pd.read_pickle("paragraphed_article_validation_bypublisher_20181122.pickle")
print(texts_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 2 columns):
id      150000 non-null object
text    150000 non-null object
dtypes: object(2)
memory usage: 2.3+ MB
None
